In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classes = base.iloc[:1400000, 28:31].values
classes = np.reshape(classes, (2000, 700, 3))
print(classes.shape)

(2000, 700, 20)
(2000, 700, 3)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNLSTM, Bidirectional, Masking

In [5]:
def criarRede():
    model = Sequential()
  
    #model.add(Masking(mask_value = 0, input_shape = (700, 20)))
  
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
  
    model.add(Dense(3, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [6]:
accu = []
precisions = []
recalls = []
f1 = []

In [7]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 100, verbose = 1, batch_size = 32)
  
    predicted = model.predict(x_test)
  
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], 3))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 3))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste

    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accu.append(balanced_accuracy_score(y_test, predicted)) 
    precisions.append(precision_score(y_test, predicted, average = 'weighted'))
    recalls.append(recall_score(y_test, predicted, average = 'weighted'))
    f1.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede()
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classes[train_index],
                           previsores[test_index], classes[test_index])

W0821 00:36:45.824917  6492 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0821 00:36:45.829909  6492 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0821 00:36:45.831899  6492 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   0    2    4 ... 1996 1997 1998] TEST: [   1    3    6    7   16   32   38   41   44   46   47   49   52   72
   76   77   94  101  105  112  127  137  141  143  147  148  153  155
  157  162  169  185  186  188  190  197  211  220  221  222  232  240
  253  256  263  273  280  282  298  299  308  310  313  315  316  320
  333  334  339  346  349  354  356  360  362  364  374  375  382  387
  391  395  410  411  419  424  425  428  435  436  447  452  454  458
  465  469  472  481  482  487  495  496  503  504  511  520  522  524
  527  530  536  538  543  544  547  559  560  571  573  574  577  580
  581  583  587  592  594  598  600  601  602  623  630  632  643  645
  648  660  665  666  671  678  685  687  690  694  698  701  704  709
  711  717  723  724  731  744  750  752  754  755  756  758  765  767
  782  783  784  785  789  791  802  806  810  817  821  825  829  834
  836  843  844  847  871  874  877  880  881  882  883  885  889  898
  905  914  919  923  928  9

W0821 00:36:49.519373  6492 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 37s 23ms/sample - loss: 0.3703 - acc: 0.6866
Epoch 2/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.3424 - acc: 0.8323
Epoch 3/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.3292 - acc: 0.8398
Epoch 4/100
1600/1600 [==============================] - 26s 17ms/sample - loss: 0.3093 - acc: 0.8565
Epoch 5/100
1600/1600 [==============================] - 26s 17ms/sample - loss: 0.3062 - acc: 0.8579
Epoch 6/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.2979 - acc: 0.8617
Epoch 7/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.2946 - acc: 0.8652
Epoch 8/100
1600/1600 [==============================] - 26s 17ms/sample - loss: 0.2937 - acc: 0.8658
Epoch 9/100
1600/1600 [==============================] - 26s 16ms/sample - loss: 0.2907 - acc: 0.8682
Epoch 10/100
1600/1600 [==============================] - 26s 17ms/sample - loss: 

1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1167 - acc: 0.9493
Epoch 81/100
1600/1600 [==============================] - 26s 17ms/sample - loss: 0.1136 - acc: 0.9482
Epoch 82/100
1600/1600 [==============================] - 26s 17ms/sample - loss: 0.1122 - acc: 0.9506
Epoch 83/100
1600/1600 [==============================] - 26s 17ms/sample - loss: 0.1107 - acc: 0.9510
Epoch 84/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1092 - acc: 0.9521
Epoch 85/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1095 - acc: 0.9513
Epoch 86/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1127 - acc: 0.9512
Epoch 87/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1131 - acc: 0.9503
Epoch 88/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1086 - acc: 0.9518
Epoch 89/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.10

...

Epoch 1/100
1600/1600 [==============================] - 29s 18ms/sample - loss: 0.3799 - acc: 0.7491
Epoch 2/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.3514 - acc: 0.8302
Epoch 3/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.3350 - acc: 0.8414
Epoch 4/100
1600/1600 [==============================] - 26s 17ms/sample - loss: 0.3184 - acc: 0.8536
Epoch 5/100
1600/1600 [==============================] - 26s 17ms/sample - loss: 0.3097 - acc: 0.8593
Epoch 6/100
1600/1600 [==============================] - 26s 17ms/sample - loss: 0.3047 - acc: 0.8617
Epoch 7/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.3015 - acc: 0.8629
Epoch 8/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.2988 - acc: 0.8648
Epoch 9/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.2969 - acc: 0.8659
Epoch 10/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 

1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1162 - acc: 0.8705
Epoch 81/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1140 - acc: 0.8347
Epoch 82/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1145 - acc: 0.8736
Epoch 83/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1141 - acc: 0.8986
Epoch 84/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1124 - acc: 0.9062
Epoch 85/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1113 - acc: 0.9033
Epoch 86/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1111 - acc: 0.9069
Epoch 87/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1132 - acc: 0.8979
Epoch 88/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1108 - acc: 0.8194
Epoch 89/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.10

1600/1600 [==============================] - 26s 17ms/sample - loss: 0.2140 - acc: 0.9079
Epoch 35/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.2102 - acc: 0.9091
Epoch 36/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.2061 - acc: 0.9115
Epoch 37/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1981 - acc: 0.9149
Epoch 38/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1971 - acc: 0.9151
Epoch 39/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1939 - acc: 0.9146
Epoch 40/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1905 - acc: 0.9178
Epoch 41/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1876 - acc: 0.9186
Epoch 42/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1841 - acc: 0.9203
Epoch 43/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.17

Epoch 1/100
1600/1600 [==============================] - 29s 18ms/sample - loss: 0.3817 - acc: 0.6889
Epoch 2/100
1600/1600 [==============================] - 26s 17ms/sample - loss: 0.3531 - acc: 0.8257
Epoch 3/100
1600/1600 [==============================] - 26s 17ms/sample - loss: 0.3404 - acc: 0.8368
Epoch 4/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.3192 - acc: 0.8528
Epoch 5/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.3081 - acc: 0.8593
Epoch 6/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.3074 - acc: 0.8593
Epoch 7/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.3020 - acc: 0.8581
Epoch 8/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.3005 - acc: 0.8625
Epoch 9/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.2958 - acc: 0.8655
Epoch 10/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 

1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1705 - acc: 0.9296
Epoch 81/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1738 - acc: 0.9261
Epoch 82/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1507 - acc: 0.9363
Epoch 83/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1301 - acc: 0.9435
Epoch 84/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1212 - acc: 0.9471
Epoch 85/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1168 - acc: 0.9497
Epoch 86/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1133 - acc: 0.9513
Epoch 87/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1116 - acc: 0.9522
Epoch 88/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.1095 - acc: 0.9529
Epoch 89/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.10

In [9]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_40 (Bidirectio (None, 700, 200)          97600     
_________________________________________________________________
dropout_40 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_41 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_41 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_42 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_42 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_43 (Bidirectio (None, 700, 200)         

In [10]:
print('Acurácias total')
print(accu)
accu = np.array(accu)
print(accu.mean())
print('Precision total')
print(precisions)
precisions = np.array(precisions)
print(precisions.mean())
print('Recalls total')
print(recalls)
recalls = np.array(recalls)
print(recalls.mean())
print('F1 total')
print(f1)
f1 = np.array(f1)
print(f1.mean())

Acurácias total
[0.6791665480447882, 0.6855690963543123, 0.6913342147390655, 0.6845891333881603, 0.6760656411580231]
0.6833449267368699
Precision total
[0.690388887845675, 0.7003783490860442, 0.6989045385758913, 0.6972015120158818, 0.6857049950445606]
0.6945156565136106
Recalls total
[0.6909075934771867, 0.7007384263077714, 0.6982352471452528, 0.6976684853004671, 0.6864895789151814]
0.6948078662291719
F1 total
[0.6905117179155797, 0.6997370685662236, 0.6985180217110422, 0.6967988073561013, 0.6856866312559341]
0.6942504493609762
